In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

# Define LSTM Model (same as training)
class LSTMModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=50, num_layers=1, output_dim=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Last time step
        return out

# Function to load model and make rolling predictions
def rolling_forecast(model_path, past_data, scaler, seq_length=24, horizon=24):
    model = LSTMModel()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Normalize past data
    scaled_past = scaler.transform(past_data.reshape(-1, 1))

    forecast = []
    current_seq = scaled_past[-seq_length:].reshape(1, seq_length, 1)

    with torch.no_grad():
        for _ in range(horizon):
            input_tensor = torch.tensor(current_seq, dtype=torch.float32)
            pred = model(input_tensor).numpy()[0][0]
            forecast.append(pred)
            # Append prediction to sequence (shift window)
            current_seq = np.append(current_seq[:, 1:, :], [[[pred]]], axis=1)

    # Inverse transform
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    return forecast.flatten()

# Load solar data
df_solar = pd.read_csv('synthetic_solar.csv')
solar_data = df_solar['solar'].values

# Load load data
df_load = pd.read_csv('synthetic_load.csv')
load_data = df_load['load'].values

# Scalers (re-fit for consistency; in practice, save during training)
solar_scaler = MinMaxScaler().fit(solar_data.reshape(-1, 1))
load_scaler = MinMaxScaler().fit(load_data.reshape(-1, 1))

# Select past 24 hours before day 30 (indices 672:695; 0-based)
past_solar = solar_data[672:696]
past_load = load_data[672:696]

# Generate forecasts
solar_forecast = rolling_forecast('solar_lstm.pth', past_solar, solar_scaler)
load_forecast = rolling_forecast('load_lstm.pth', past_load, load_scaler)

# Save to CSV for MATLAB import
pd.DataFrame({'solar_forecast': solar_forecast}).to_csv('solar_forecast_day30.csv', index=False)
pd.DataFrame({'load_forecast': load_forecast}).to_csv('load_forecast_day30.csv', index=False)

print("Forecasts saved to CSVs for day 30.")

Forecasts saved to CSVs for day 30.
